In [1]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48.4 MB 11.2 MB/s 


In [2]:
import os
import shutil
import numpy as np
import sys
# sys.path.append('./')
# import config_training
#from config_training import config 
# config = config_training.config
from scipy.io import loadmat
import numpy as np
import h5py
import pandas
import scipy
from scipy.ndimage.interpolation import zoom
from skimage import measure
import SimpleITK as sitk
from scipy.ndimage.morphology import binary_dilation,generate_binary_structure
from skimage.morphology import convex_hull_image
import pandas
from multiprocessing import Pool
from functools import partial

import warnings
from matplotlib import pyplot as plt

In [3]:
config = {'train_data_path':['/content/drive/MyDrive/CapastoneProject/data/luna16/subset0/'
                             ],
          'val_data_path':['/content/drive/MyDrive/CapastoneProject/data/luna16/subset1/'], 
          'test_data_path':['/content/drive/MyDrive/CapastoneProject/data/luna16/subset2/'], 
          
          'train_preprocess_result_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/preprocess/',
          'val_preprocess_result_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/preprocess/',  
          'test_preprocess_result_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/preprocess/',
          
          'train_annos_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/CSVFILES/annotations.csv',
          'val_annos_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/CSVFILES/annotations.csv',
          'test_annos_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/CSVFILES/annotations.csv',

          'black_list':[],
          
          'preprocessing_backend':'python',

          'luna_segment':'/content/drive/MyDrive/CapastoneProject/data/luna16/seg-lungs-LUNA16/',
          'preprocess_result_path':'/content/drive/MyDrive/CapastoneProject/data/luna16/preprocess/',
          'luna_data':'/content/drive/MyDrive/CapastoneProject/data/luna16/',
          'luna_label':'/content/drive/MyDrive/CapastoneProject/data/luna16/CSVFILES/annotations.csv'}

In [4]:
#def preprocess_luna():
luna_segment = config['luna_segment']
savepath = config['preprocess_result_path']
luna_data = config['luna_data']
luna_label = config['luna_label']
finished_flag = False

In [5]:
def load_itk_image(filename):
  with open(filename) as f:
      contents = f.readlines()
      f.close()
      line = [k for k in contents if k.startswith('TransformMatrix')][0]
      transformM = np.array(line.split(' = ')[1].split(' ')).astype('float')
      transformM = np.round(transformM)
      if np.any( transformM!=np.array([1,0,0, 0, 1, 0, 0, 0, 1])):
          isflip = True
      else:
          isflip = False

  itkimage = sitk.ReadImage(filename)
  numpyImage = sitk.GetArrayFromImage(itkimage)
    
  numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
  numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    
  return numpyImage, numpyOrigin, numpySpacing,isflip


In [6]:
def process_mask(mask):
    convex_mask = np.copy(mask)
    for i_layer in range(convex_mask.shape[0]):
        mask1  = np.ascontiguousarray(mask[i_layer])
        if np.sum(mask1)>0:
            mask2 = convex_hull_image(mask1)
            if np.sum(mask2)>1.5*np.sum(mask1):
                mask2 = mask1
        else:
            mask2 = mask1
        convex_mask[i_layer] = mask2
    struct = generate_binary_structure(3,1)  
    dilatedMask = binary_dilation(convex_mask,structure=struct,iterations=10) 
    return dilatedMask

In [7]:
def lumTrans(img):
    lungwin = np.array([-1200.,600.])
    newimg = (img-lungwin[0])/(lungwin[1]-lungwin[0])
    newimg[newimg<0]=0
    newimg[newimg>1]=1
    newimg = (newimg*255).astype('uint8')
    return newimg

In [8]:
def resample(imgs, spacing, new_spacing,order=2):
    if len(imgs.shape)==3:
        new_shape = np.round(imgs.shape * spacing / new_spacing)
        true_spacing = spacing * imgs.shape / new_shape
        resize_factor = new_shape / imgs.shape
        imgs = zoom(imgs, resize_factor, mode = 'nearest',order=order)
        return imgs, true_spacing
    elif len(imgs.shape)==4:
        n = imgs.shape[-1]
        newimg = []
        for i in range(n):
            slice = imgs[:,:,:,i]
            newslice,true_spacing = resample(slice,spacing,new_spacing)
            newimg.append(newslice)
        newimg=np.transpose(np.array(newimg),[1,2,3,0])
        return newimg,true_spacing
    else:
        raise ValueError('wrong shape')

In [9]:
def worldToVoxelCoord(worldCoord, origin, spacing):
     
    stretchedVoxelCoord = np.absolute(worldCoord - origin)
    voxelCoord = stretchedVoxelCoord / spacing
    return voxelCoord

In [10]:
def savenpy_luna(id, annos, filelist, luna_segment, luna_data,savepath):
  islabel = True
  isClean = True
  resolution = np.array([1,1,1])
  #     resolution = np.array([2,2,2])
  name = filelist[id]
  print(name)
  sliceim,origin,spacing,isflip = load_itk_image(os.path.join(luna_data,name+'.mhd'))

  Mask,origin,spacing,isflip = load_itk_image(os.path.join(luna_segment,name+'.mhd'))
  if isflip:
      Mask = Mask[:,::-1,::-1]
  newshape = np.round(np.array(Mask.shape)*spacing/resolution).astype('int')
  m1 = Mask==3
  m2 = Mask==4
  Mask = m1+m2

  xx,yy,zz= np.where(Mask)
  box = np.array([[np.min(xx),np.max(xx)],[np.min(yy),np.max(yy)],[np.min(zz),np.max(zz)]])
  box = box*np.expand_dims(spacing,1)/np.expand_dims(resolution,1)
  box = np.floor(box).astype('int')
  margin = 5
  extendbox = np.vstack([np.max([[0,0,0],box[:,0]-margin],0),np.min([newshape,box[:,1]+2*margin],axis=0).T]).T

  this_annos = np.copy(annos[annos[:,0]==(name)])        

  if isClean:
      convex_mask = m1
      dm1 = process_mask(m1)
      dm2 = process_mask(m2)
      dilatedMask = dm1+dm2
      Mask = m1+m2

      extramask = dilatedMask ^ Mask
      bone_thresh = 210
      pad_value = 170

      if isflip:
          sliceim = sliceim[:,::-1,::-1]
          print('flip!')
      sliceim = lumTrans(sliceim)
      sliceim = sliceim*dilatedMask+pad_value*(1-dilatedMask).astype('uint8')
      bones = (sliceim*extramask)>bone_thresh
      sliceim[bones] = pad_value
      
      sliceim1,_ = resample(sliceim,spacing,resolution,order=1)
      sliceim2 = sliceim1[extendbox[0,0]:extendbox[0,1],
                  extendbox[1,0]:extendbox[1,1],
                  extendbox[2,0]:extendbox[2,1]]
      sliceim = sliceim2[np.newaxis,...]
      

  if islabel:
      this_annos = np.copy(annos[annos[:,0]==(name)])
      print('this_annos',this_annos)
      print(type(this_annos))
      label = []
      if len(this_annos)>0:
          for c in this_annos:
              print('c',c)
              print('c[1:4]',c[1:4])
              pos = worldToVoxelCoord(c[1:4][::-1],origin=origin,spacing=spacing)
              print(pos)
              try:
                if isflip:
                    pos[1:] = Mask.shape[1:3]-pos[1:]
              except Exception as e:
                print(e)
              try:
                
                label.append(np.concatenate([pos,[c[4]/spacing[1]]]))
                print(label)
              except Exception as e:
                print(e)
                
          
      label = np.array(label)
      if len(label)==0:
          label2 = np.array([[0,0,0,0]])
      else:
          label2 = np.copy(label).T
          label2[:3] = label2[:3]*np.expand_dims(spacing,1)/np.expand_dims(resolution,1)
          label2[3] = label2[3]*spacing[1]/resolution[1]
          label2[:3] = label2[:3]-np.expand_dims(extendbox[:,0],1)
          label2 = label2[:4].T
          np.save(os.path.join(savepath,name+'_label.npy'), label2)
          np.save(os.path.join(savepath, name+'_clean.npy'), sliceim)
          np.save(os.path.join(savepath, name+'_spacing.npy'), spacing)
          np.save(os.path.join(savepath, name+'_extendbox.npy'), extendbox)
          np.save(os.path.join(savepath, name+'_origin.npy'), origin)
          np.save(os.path.join(savepath, name+'_mask.npy'), Mask)
          print('saved',name)

In [12]:
luna_segment = config['luna_segment']
savepath = config['preprocess_result_path']
luna_data = config['luna_data']
luna_label = config['luna_label']
finished_flag = '.flag_preprocessluna'
print('starting preprocessing luna')
finished_flag = False
if not finished_flag:
    annos = np.array(pandas.read_csv(luna_label))
    pool = Pool()
    if not os.path.exists(savepath):
        os.mkdir(savepath)
    for setidx in range(10):
        print('process subset', setidx)
        filelist = [f.split('.mhd')[0] for f in os.listdir(luna_data+'subset'+str(setidx)) if f.endswith('.mhd') ]
        print('Number of files', len(filelist))
        if not os.path.exists(savepath+'subset'+str(setidx)):
            os.mkdir(savepath+'subset'+str(setidx))
        partial_savenpy_luna = partial(savenpy_luna, annos=annos, filelist=filelist,
                                    luna_segment=luna_segment, luna_data=luna_data+'subset'+str(setidx)+'/', 
                                    savepath=savepath+'subset'+str(setidx)+'/')
        N = len(filelist)
        #savenpy(1)
        _=pool.map(partial_savenpy_luna,range(N))
    pool.close()
    pool.join()
print('end preprocessing luna')

starting preprocessing luna
1.3.6.1.4.1.14519.5.2.1.6279.6001.137763212752154081977261297097
1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260
process subset 0
Number of files 89
this_annos []
<class 'numpy.ndarray'>
1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492
this_annos [['1.3.6.1.4.1.14519.5.2.1.6279.6001.137763212752154081977261297097'
  -103.723003 153.3927131 -465.845305 11.13983165]
 ['1.3.6.1.4.1.14519.5.2.1.6279.6001.137763212752154081977261297097'
  -80.44626828 183.9384127 -467.8757027 17.69382414]]
<class 'numpy.ndarray'>
c ['1.3.6.1.4.1.14519.5.2.1.6279.6001.137763212752154081977261297097'
 -103.723003 153.3927131 -465.845305 11.13983165]
c[1:4] [-103.723003 153.3927131 -465.845305]
[91.654695 307.9719000926316 81.8794698105263]
[array([91.654695, 307.9719000926316, 81.8794698105263, 15.00945738105263],
      dtype=object)]
c ['1.3.6.1.4.1.14519.5.2.1.6279.6001.137763212752154081977261297097'
 -80.44626828 183.9384127 -467.8757027 17.6938

FileNotFoundError: ignored